In [99]:
# pip install ta

In [100]:
from typing import Union, Dict, Set, List, TypedDict, Annotated
import yfinance as yf
import datetime as dt
from langchain_core.tools import tool
import traceback
import pandas as pd
import dotenv
import os
dotenv.load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [101]:
import yfinance as yf
import pandas as pd
import datetime as dt

# ----------------------------------------
# 1. Parámetros y lista de acciones IPSA
# ----------------------------------------
start_date = dt.datetime.now() - dt.timedelta(weeks=12)  # 3 meses atrás
end_date = dt.datetime.now()

ipsa_stocks = [
    "SQM-B.SN", "CHILE.SN", "BSANTANDER.SN", "COPEC.SN", "ENELAM.SN", "CENCOSUD.SN",
    "CMPC.SN", "BCI.SN", "FALABELLA.SN", "ENELCHILE.SN", "PARAUCO.SN", "COLBUN.SN",
    "CCU.SN", "ANDINA-B.SN", "VAPORES.SN", "AGUAS-A.SN", "QUINENCO.SN", "CENCOMALLS.SN",
    "LTM.SN", "CONCHATORO.SN", "ENTEL.SN", "CAP.SN", "ORO-BLANCO.SN", "MALLPLAZA.SN",
    "ECL.SN", "IAM.SN", "SMU.SN", "ITAUCL.SN", "SONDA.SN", "RIPLEY.SN"
]

# ----------------------------------------
# 2. Descarga de datos en un solo request
# ----------------------------------------
print("Descargando datos de Yahoo Finance para todas las acciones del IPSA...")
all_data = yf.download(
    tickers=ipsa_stocks,
    start=start_date,
    end=end_date,
    interval='1d',
    group_by='ticker',  # Devuelve un dict { TICKER: DF }
    auto_adjust=False,
    threads=False  # evita multi-threading para mayor estabilidad
)

print("\nDescarga completa. Comenzando análisis...\n")

# ----------------------------------------
# 3. Funciones auxiliares
# ----------------------------------------
def calculate_rsi(series, period=14):
    """Calcula el RSI de una serie de precios de cierre."""
    delta = series.diff().dropna()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

@tool
def get_financial_metrics(ticker: str) -> Union[Dict, str]:
    """Fetches key financial ratios for a given ticker."""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'pe_ratio': info.get('forwardPE'),
            'price_to_book': info.get('priceToBook'),
            'debt_to_equity': info.get('debtToEquity'),
            'profit_margins': info.get('profitMargins')
        }
    except Exception as e:
        return f"Error fetching ratios: {str(e)}"
    
def safe_get_financial_metrics(ticker):
    """Obtiene métricas financieras de forma segura (manejo de string-errores)."""
    data = get_financial_metrics(ticker)
    if isinstance(data, str):
        # Hubo error
        return {
            'pe_ratio': None,
            'debt_to_equity': None
        }
    return data

def safe_get_financial_metrics(ticker):
    """Obtiene métricas financieras de forma segura (manejo de string-errores)."""
    data = get_financial_metrics(ticker)
    if isinstance(data, str):
        # Hubo error
        return {
            'pe_ratio': None,
            'debt_to_equity': None
        }
    return data

# ----------------------------------------
# 4. Bucle principal de análisis
# ----------------------------------------
results = []
for ticker in ipsa_stocks:
    print(f"--- Análisis para {ticker} ---")

    # 4.1) Verificamos si se descargó algo para este ticker
    #     all_data[ticker] será un DF con columnas: [Open, High, Low, Close, Adj Close, Volume]
    #     o puede no existir si no se devolvieron datos
    if ticker not in all_data:
        print(f"   ❌ No hay datos descargados para {ticker} (posible error con el sufijo .SN)")
        results.append({
            "Stock": ticker,
            "Último Precio": None,
            "RSI": None,
            "P/E Ratio": None,
            "Debt/Equity": None
        })
        continue

    df = all_data[ticker].copy()

    # 4.2) Validamos que tenga filas
    if df.empty:
        print(f"   ⚠️ DataFrame vacío para {ticker}.")
        results.append({
            "Stock": ticker,
            "Último Precio": None,
            "RSI": None,
            "P/E Ratio": None,
            "Debt/Equity": None
        })
        continue

    # 4.3) Asegurar que las columnas tengan el formato [Open, High, Low, Close, Volume, etc.]
    if "Close" not in df.columns:
        print(f"   ⚠️ La columna 'Close' no está presente en {ticker}.")
        results.append({
            "Stock": ticker,
            "Último Precio": None,
            "RSI": None,
            "P/E Ratio": None,
            "Debt/Equity": None
        })
        continue

    # 4.4) Calculamos RSI
    #     - Si no hay suficientes filas, RSI puede salir mayoritariamente NaN
    df["RSI"] = calculate_rsi(df["Close"])

    # 4.5) Tomamos el último precio y el último RSI
    last_close = df["Close"].iloc[-1]
    last_rsi = df["RSI"].iloc[-1] if not df["RSI"].isna().all() else None

    # 4.6) Buscamos métricas financieras
    fm = safe_get_financial_metrics(ticker)  # P/E, Debt/Equity, etc.
    pe_ratio = fm.get('pe_ratio')
    debt_to_equity = fm.get('debt_to_equity')

    # 4.7) Armamos registro
    stock_data = {
        "Stock": ticker,
        "Último Precio": last_close,
        "RSI": round(last_rsi, 2) if last_rsi is not None else None,
        "P/E Ratio": pe_ratio,
        "Debt/Equity": debt_to_equity
    }

    # 4.8) Calculamos porcentaje de NaN
    nan_count = sum(v is None for v in stock_data.values())
    nan_percentage = (nan_count / len(stock_data)) * 100

    # 4.9) Mostramos en consola un resumen inmediato
    print(f"   Precio: {stock_data['Último Precio']}, RSI: {stock_data['RSI']}, "
          f"P/E: {stock_data['P/E Ratio']}, D/E: {stock_data['Debt/Equity']}")
    print(f"   {nan_percentage:.2f}% de valores faltantes en {ticker}.\n")

    # 4.10) Agregamos a la lista de resultados
    results.append(stock_data)

# ----------------------------------------
# 5. DataFrame final y reporte
# ----------------------------------------
df_results = pd.DataFrame(results)

print("=== Resumen final del análisis ===\n")
# Ordenamos por RSI (ascendente)
df_results = df_results.sort_values(by="RSI", ascending=True, na_position='last')

# Reemplazar None con NaN para mejor visualización
df_results.fillna(pd.NA, inplace=True)

print(df_results)


Descargando datos de Yahoo Finance para todas las acciones del IPSA...


[*********************100%***********************]  30 of 30 completed



Descarga completa. Comenzando análisis...

--- Análisis para SQM-B.SN ---
   Precio: 39399.0, RSI: 56.68, P/E: 27.958418, D/E: 91.006
   0.00% de valores faltantes en SQM-B.SN.

--- Análisis para CHILE.SN ---
   Precio: 120.5, RSI: 81.58, P/E: 10.79841, D/E: None
   20.00% de valores faltantes en CHILE.SN.

--- Análisis para BSANTANDER.SN ---
   Precio: 49.400001525878906, RSI: 72.25, P/E: 9.471985, D/E: None
   20.00% de valores faltantes en BSANTANDER.SN.

--- Análisis para COPEC.SN ---
   Precio: 6620.0, RSI: 65.12, P/E: 7450.0054, D/E: 70.352
   0.00% de valores faltantes en COPEC.SN.

--- Análisis para ENELAM.SN ---
   Precio: 87.0, RSI: 33.18, P/E: 8.8669195, D/E: 32.236
   0.00% de valores faltantes en ENELAM.SN.

--- Análisis para CENCOSUD.SN ---
   Precio: 2380.0, RSI: 71.83, P/E: 10.787469, D/E: 113.837
   0.00% de valores faltantes en CENCOSUD.SN.

--- Análisis para CMPC.SN ---
   Precio: 1675.0, RSI: 69.03, P/E: 8666.632, D/E: 72.507
   0.00% de valores faltantes en CMPC.S

In [102]:
from typing import TypedDict, Annotated
import pandas as pd
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 1) Creamos un "estado" que guarde mensajes y el DataFrame
class IPSAState(TypedDict):
    messages: Annotated[list, add_messages]
    df: pd.DataFrame  # Aquí guardaremos el DF final con datos de las acciones

# 2) Instanciamos el grafo
graph_builder = StateGraph(IPSAState)

# 3) Creamos nuestro LLM de LangChain
llm = ChatOpenAI(model='gpt-4o-mini', api_key=openai_api_key, temperature=0)

In [103]:
def generate_ipsa_analysis(state: IPSAState):
    """
    Toma el DataFrame del estado, lo convierte a CSV,
    y construye un prompt para el LLM pidiendo una recomendación.
    """
    df = state['df']
    
    # Convertir a CSV para que el LLM tenga datos en texto
    data_csv = df.to_csv(index=False)
    
    # Preparamos mensajes para el LLM
    system_prompt = """
    Eres un analista financiero experto en la bolsa chilena.
    Tu tarea es examinar la información de múltiples acciones 
    (último precio, RSI, P/E, etc.) y dar un análisis/recomendación 
    fundamentado, en español, sin inventar datos.
    """
    
    human_prompt = f"""
    Aquí tienes datos de varias acciones del IPSA en formato CSV:
    ```
    {data_csv}
    ```
    Por favor, proporciona:
    1) Un resumen de los puntos clave (ej: rangos de RSI, P/E, etc.).
    2) Fortalezas o debilidades que observes.
    3) Una recomendación objetiva basada en los datos.
    4) Una lista de cinco acciones recomendadas. Sòlamente el nombre, sin mayores explicaciones.
    """

    # Creamos la lista de mensajes
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=human_prompt)
    ]

    # Invocamos el LLM
    response = llm.invoke(messages)

    # Retornamos las respuestas
    # (generalmente en un dict con 'messages', para que LangGraph lo maneje)
    # Si deseas concatenar con mensajes anteriores, hazlo aquí.
    return {
        'messages': state['messages'] + [response] 
    }


In [104]:
# 5) Registramos el nodo con nombre NUEVO (para evitar conflictos)
graph_builder.add_node('analyze_ipsa_df', generate_ipsa_analysis)
# 6) Conectamos el flujo START -> analyze_ipsa_df -> END
graph_builder.add_edge(START, 'analyze_ipsa_df')
graph_builder.add_edge('analyze_ipsa_df', END)

# 7) Compilamos el grafo
ipsa_graph = graph_builder.compile()

In [ ]:
# Por ejemplo, un mensaje inicial del usuario
initial_messages = [("user", "Recomienda un portafolio para este conjunto de acciones.")]

# Ejecutamos el grafo
state_input = {
    "messages": initial_messages,
    "df": df_results  # <-- tu DataFrame ya procesado
}

graph = graph_builder.compile()
events = graph.stream(state_input, stream_mode='values')

for event in events:
    # Cada 'event' es un dict parcial con estados o mensajes
    if 'messages' in event:
        final_messages = event['messages']
        # Podrías ir imprimiendo parcial o esperar el final
        print(final_messages[-1].content)


Recomienda un portafolio para este conjunto de acciones.
### Resumen de Puntos Clave

1. **Rango de RSI**:
   - La mayoría de las acciones se encuentran en un rango de RSI entre 33.18 y 96.01.
   - Acciones con RSI por debajo de 40 (potencialmente sobrevendidas): ENELAM.SN, CENCOMALLS.SN, VAPORES.SN.
   - Acciones con RSI por encima de 70 (potencialmente sobrecompradas): SMU.SN (96.01), ITAUCL.SN (81.21), FALABELLA.SN (79.55).

2. **Rango de P/E Ratio**:
   - P/E ratios muy variados, desde valores extremadamente bajos como VAPORES.SN (8.70) hasta valores muy altos como COLBUN.SN (6703.47) y ECL.SN (5168.28).
   - La mayoría de las acciones tienen un P/E entre 6 y 20, lo que puede indicar un valor razonable.

3. **Debt/Equity**:
   - La relación deuda/capital varía considerablemente, con algunas empresas como VAPORES.SN mostrando una relación muy baja (0.019), lo que indica una baja dependencia de deuda.
   - Otras, como ANDINA-B.SN y ECL.SN, tienen ratios muy altos, lo que puede ser un

In [108]:
output_dict = {
  "data": df_results.to_dict(orient='records'),
  "analysis": final_messages[-1].content
}

import json
with open("results.json", "w", encoding="utf-8") as f:
    json.dump(output_dict, f, ensure_ascii=False, indent=2)